### Logistic Reg Model - Binary Classifier


In [1]:
# IDEAS: 
# https://stackoverflow.com/questions/38077190/how-to-increase-the-model-accuracy-of-logistic-regression-in-scikit-python

# scale the columns
# https://www.keboola.com/blog/logistic-regression-machine-learning
# https://www.kdnuggets.com/2020/04/data-transformation-standardization-normalization.html
# drop features/remove multicoliniarity features

# look to use class weight/class imbalances

# try SVM; maybe we cannot linearly seperate yes vs no leads column?

# add features???

In [143]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd
import numpy as np

# Import and read the csv.
#df = pd.read_csv('model_df1_official.csv', index_col=[0])
# OR: 
df = pd.read_csv('nnML_tiers.csv', index_col=[0])
df.head()

,state,brand,spend,platform,date_start,Impressions,leads,link_clicks,reach,agency,State Tiers,Agency Tiers
770,CA,B1,7.28,Facebook Ads,5/11/2021,427,0.0,2,403.0,Agency 4,4,4
771,CA,B1,13.32,Facebook Ads,6/23/2021,1042,1.0,7,962.0,Agency 4,4,4
772,CA,B1,10.11,Facebook Ads,6/13/2021,337,0.0,0,321.0,Agency 4,4,4
773,CA,B1,0.00,Facebook Ads,4/20/2021,0,1.0,0,0.0,Agency 4,4,4
774,CA,B1,7.35,Facebook Ads,5/11/2021,379,1.0,0,364.0,Agency 4,4,4


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157672 entries, 770 to 160484
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   state         157672 non-null  object 
 1   brand         157672 non-null  object 
 2   spend         157672 non-null  float64
 3   platform      157672 non-null  object 
 4   date_start    157672 non-null  object 
 5   Impressions   157672 non-null  int64  
 6   leads         157672 non-null  float64
 7   link_clicks   157672 non-null  int64  
 8   reach         157672 non-null  float64
 9   agency        157672 non-null  object 
 10  State Tiers   157672 non-null  int64  
 11  Agency Tiers  157672 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 15.6+ MB


In [145]:
df['state']

770       CA
771       CA
772       CA
773       CA
774       CA
          ..
160480    ME
160481    ME
160482    ME
160483    ME
160484    ME
Name: state, Length: 157672, dtype: object

In [146]:
# version to look at only CA
# observe column brand
df_ca = df.loc[df['state']== 'VA']
# output shows CA starts (index) at 770 and ends at 146479

In [147]:
# Drop all CA values from state column:
df_ca.head()

,state,brand,spend,platform,date_start,Impressions,leads,link_clicks,reach,agency,State Tiers,Agency Tiers
11751,VA,B1,17.43,Facebook Ads,12/20/2021,1275,1.0,2,1197.0,Agency 4,3,4
11752,VA,B2,3.89,Facebook Ads,9/7/2021,194,1.0,0,187.0,Agency 4,3,4
11753,VA,B2,4.00,Facebook Ads,7/21/2021,142,1.0,1,141.0,Agency 4,3,4
11754,VA,B2,29.31,Facebook Ads,9/29/2021,1779,1.0,4,1696.0,Agency 4,3,4
11755,VA,B2,5.28,Facebook Ads,8/5/2021,332,1.0,0,328.0,Agency 4,3,4


In [154]:
# df2 = df.drop(columns=['state', 'agency', 'brand', 'platform', 'date_start', 'Impressions'])
# df2

# OR ca option
df_ca1 = df_ca.drop(columns=['state', 'agency', 'brand', 'platform', 'date_start', 'link_clicks'])
df_ca1.head()

,spend,Impressions,leads,reach,State Tiers,Agency Tiers
11751,17.43,1275,1.0,1197.0,3,4
11752,3.89,194,1.0,187.0,3,4
11753,4.00,142,1.0,141.0,3,4
11754,29.31,1779,1.0,1696.0,3,4
11755,5.28,332,1.0,328.0,3,4


In [155]:
# steps: 
# create the model with LogisticRegression()
# train the model with model.fit()
# make predictions with model.predict()
# validate with accuracy_score()

### Hyperparam check

In [156]:
logModel = LogisticRegression()

In [157]:
param_grid = [
    {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': np.logspace(-4,4,20),
    'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
    'max_iter': [100,1000,2500,5000]}
]


In [158]:
# test for most optimal params 
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(logModel, param_grid = param_grid, cv=3, verbose=True, n_jobs=-1)

In [159]:
best_clf = clf.fit(X,y)

Fitting 3 folds for each of 1600 candidates, totalling 4800 fits


C:\Users\dia78039\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.62068509 ...        nan 0.62675259 0.62675259]
  category=UserWarning


In [160]:
best_clf.best_estimator_
# shows that I need a penalty of l1, solver type saga, and c value of .0001

LogisticRegression(C=0.0001, penalty='l1', solver='saga')

### Seperate features, x and target, y

In [161]:
# y = df2["leads"]
# X = df2.drop(columns="leads")

# or CA option
y = df_ca1["leads"]
X = df_ca1.drop(columns="leads")

### Split into train and test features

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(1979, 5)

### Create Log Reg Model 

In [163]:
# edit the params as needed
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='saga',
                                C=0.0001,
                                max_iter=500,
                                random_state=1,
                                class_weight='balanced',
                               multi_class='auto', 
                               penalty='l1')

### Fit/train or model using training data

In [164]:
classifier.fit(X_train, y_train)

LogisticRegression(C=0.0001, class_weight='balanced', max_iter=500,
                   penalty='l1', random_state=1, solver='saga')

### Make predictions

In [165]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,1.0,0.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,1.0
6,1.0,0.0
7,1.0,1.0
8,1.0,0.0
9,1.0,1.0


### Validate

In [166]:
# accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.6242424242424243


In [167]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)
# TP = 0
# FP = 0
# FN = 17,460
# TN = 21,958

# precision = TP/(TP+FP), in this case a precision of 0 means what?
# sensitivity = TP/(TP+FN), in this case a score of 0 means what? = recall

[[ 18 228]
 [ 20 394]]


In [168]:
# classification report
report = classification_report(y_test, y_pred)
print(report)

### note: 
# ytest are the outcomes, either yes or no on leads column target 
# y_pred are the predictions
# do we read the accuracy score as 56% on predictions as a lead? 
# F1 = harmonic mean, takes sensitivity and precision = .72 here means what?

              precision    recall  f1-score   support

         0.0       0.47      0.07      0.13       246
         1.0       0.63      0.95      0.76       414

    accuracy                           0.62       660
   macro avg       0.55      0.51      0.44       660
weighted avg       0.57      0.62      0.52       660

